In [1]:
from pathlib import Path
from pathlib import *
import pandas as pd
import re
from random import randint
import spacy
from time import time
import matplotlib.pyplot as plt
import gensim
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import TfidfModel
from gensim.models import CoherenceModel
from gensim.test.utils import datapath
from gensim import  models
nlpd = spacy.load("da_core_news_lg")
import os

In [2]:
path = "../../Folketingstidende/Data_2009_2016/"
dir_path = Path(f"{path}")
csv_files = list(dir_path.glob("*.csv"))

In [4]:
csv_files.sort()
csv_files

[PosixPath('../../Folketingstidende/Data_2009_2016/20091.csv'),
 PosixPath('../../Folketingstidende/Data_2009_2016/20101.csv'),
 PosixPath('../../Folketingstidende/Data_2009_2016/20102.csv'),
 PosixPath('../../Folketingstidende/Data_2009_2016/20111.csv'),
 PosixPath('../../Folketingstidende/Data_2009_2016/20121.csv'),
 PosixPath('../../Folketingstidende/Data_2009_2016/20131.csv'),
 PosixPath('../../Folketingstidende/Data_2009_2016/20141.csv'),
 PosixPath('../../Folketingstidende/Data_2009_2016/20142.csv'),
 PosixPath('../../Folketingstidende/Data_2009_2016/20151.csv'),
 PosixPath('../../Folketingstidende/Data_2009_2016/20161.csv')]

In [19]:
folketingsår = pd.read_csv(csv_files[0])
folketingsår

,ID,Session,Date,Start time,End time,Time,Agenda item,Case no,Case type,Agenda title,Subject 1,Subject 2,Name,Gender,Party,Role,Title,Birth,Age,Text
0,20091006120032,20091,2009-10-06,12:00:32,12:02:08,96,2009-10-06-0,0,FM,Punkt 0,other,none,Niels Helveg Petersen,Male,RV,aldersformanden,Aldersformanden,1939-01-17,70,Mødet er åbnet. I henhold til grundloven er Fo...
1,20091006120208,20091,2009-10-06,12:02:08,12:04:01,113,2009-10-06-0,0,FM,Punkt 0,other,none,Thor Pedersen,Male,V,formand,Formanden,1945-06-14,64,"Jeg vil gerne takke for den tillid, som Tinget..."
2,20091006120401,20091,2009-10-06,12:04:01,12:50:39,2798,2009-10-06-0,0,FM,Punkt 0,other,none,Lars Løkke Rasmussen,Male,V,minister,Statsministeren,1964-05-15,45,Danmark er et godt land at leve i. Vi har bygg...
3,20091006125039,20091,2009-10-06,12:50:39,12:51:10,31,2009-10-06-0,0,FM,Punkt 0,other,none,Thor Pedersen,Male,V,formand,Formanden,1945-06-14,64,Tak til statsministeren. Jeg kan oplyse Folket...
4,20091007130015,20091,2009-10-07,13:00:15,13:02:41,146,2009-10-07-0,0,FM,Punkt 0,other,none,Thor Pedersen,Male,V,formand,Formanden,1945-06-14,64,Mødet er åbnet. Transportministeren (Lars Barf...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59851,20100907200835,20091,2010-09-07,20:08:35,20:09:29,54,2010-09-07-3,225,L,1. behandling af L 225: Om finanslov for finan...,Economy,none,Rasmus Prehn,Male,S,medlem,nil,1973-06-18,37,"Det er jo sådan set lidt rystende, at vi har e..."
59852,20100907200929,20091,2010-09-07,20:09:29,20:09:33,4,2010-09-07-3,225,L,1. behandling af L 225: Om finanslov for finan...,Economy,none,Karen J. Klint,Female,S,formand,Den fg. formand,1947-10-06,63,Værsgo til ministeren.
59853,20100907200933,20091,2010-09-07,20:09:33,20:09:47,14,2010-09-07-3,225,L,1. behandling af L 225: Om finanslov for finan...,Economy,none,Claus Hjort Frederiksen,Male,V,minister,Finansministeren,1947-09-04,63,Til hr. Rasmus Prehn bliver jeg bare nødt til ...
59854,20100907200947,20091,2010-09-07,20:09:47,20:10:06,19,2010-09-07-3,225,L,1. behandling af L 225: Om finanslov for finan...,Economy,none,Karen J. Klint,Female,S,formand,Den fg. formand,1947-10-06,63,"Er der flere, der ønsker ordet? Da der ikke er..."


In [16]:
EU_ord = [" eu ", " ef ", "europæiske fællesskab", "europæiske union", "fællesmarked", "ømu", "maastricht", 
          "amsterdamtraktaten", "eu-", "ef-", "edinburgh", "indre marked", "lissabon", "grækenland", " euro ", "euro-"]

In [21]:
Folketingsår_opdelt_per_taler_2009_2016 = {}
for år in csv_files:
    data_ft = pd.read_csv(folketingsår)
    år_tale = [tale for tale in data_ft["Text"]]
    år_taler = [taler for taler in data_ft["Name"]]
    parti = [parti for parti in data_ft["Party"]]
    Folketingsår_opdelt_per_taler_2009_2016[år] = []
    Folketingsår_opdelt_per_taler_2009_2016[år].append(år_tale)
    Folketingsår_opdelt_per_taler_2009_2016[år].append(år_talere)
    Folketingsår_opdelt_per_taler_2009_2016[år].append(år_parti)

NameError: name 'år_talere' is not defined